In [124]:
#Thought Process:
# 1) Retrieve a route # and trip_id from gtsf data.
# 2) Filter APC data with route #, direction_id, range of departure time
# 3) For every trip in filtered APC, compare difference to official gtsf schedule. 
# 4) Calculate average wait time by (a) finding difference between official and scheduled bus time, 
#                                   (b) summing + dividing by number of bus stops in particular trip


#Summary of this Jupyter Notebook:
# 1) I used Adrianna's/Anthony's code on Slack to comb through 03/14/2018 Dropbox Folder and extracted Route 5, Trip 6025741
# 2) I attempted to determine direction by calculating longitude/latitude distance between 1st and last gtsf stops
#    Sadly, it didn't work. I need help figuring how to do determine direction without utilizing Google Maps/ Tableau
# 3) I only used 2,000,000 rows from APC.csv but someone with a better computer please try my code on the rest of the data.
# 4) I eliminated APC rows by filtering for a range of departure time, route number, and direction. 
# 5) For every vehicle in this new APC, loop through their trips.
# 6) 

In [125]:
#Adrianna's code on Slack

import pandas as pd
ftrips = pd.read_csv("trips.txt", delimiter=",")
froutes = pd.read_csv("routes.txt", delimiter=",")
fstop_times = pd.read_csv("stop_times.txt", delimiter=",")
fstops = pd.read_csv("stops.txt", delimiter=",")

In [126]:
#Anthony's code on Slack
#I used files from 03/14/2018 because there were MARTA updated policies in April 2018 (?)

trips = ftrips
stop_times = fstop_times
stops = fstops
routes = froutes
#trips = trips[trips.service_id == service_id]
trips = trips.merge(routes[['route_id', 'route_short_name']], on=['route_id'])
route_stops = stop_times.merge(trips[['trip_id', 'direction_id', 'block_id', 'route_short_name', 'service_id']],  # removed route_id **
                               on=['trip_id'])
route_stops = route_stops.merge(stops[['stop_id', 'stop_lat', 'stop_lon']])

In [127]:
#I wanted to see the column names in the final combined gtsf
print(route_stops.columns)

Index(['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence',
       'direction_id', 'block_id', 'route_short_name', 'service_id',
       'stop_lat', 'stop_lon'],
      dtype='object')


In [128]:
#extract rows that had route_short_name = 5
route_5 = route_stops.loc[(route_stops['route_short_name'] == "5")] 

In [129]:
route_5.trip_id.unique() #print unique trip_id's for all of route 5 and select one
#I chose trip 6025741

array([6025741, 6025742, 6025743, 6026021, 6026024, 6026031, 6026032,
       6026033, 6026034, 6026035, 6026036, 6026132, 6026133, 6026134,
       6026135, 6026176, 6026177, 6026178, 6026179, 6026180, 6026181,
       6026184, 6026185, 6026242, 6026243, 6026244, 6026245, 6026288,
       6026289, 6026290, 6027190, 6027191, 6027192, 6027193, 6027194,
       6027195, 6027196, 6028466, 6031274, 6031279, 6031280, 6031281,
       6031282, 6031287, 6031288, 6031289, 6031290, 6031291, 6031382,
       6031383, 6031384, 6031430, 6031432, 6031433, 6031434, 6031435,
       6032078, 6032079, 6032080, 6032081, 6032118, 6032119, 6032120,
       6032182, 6032241, 6032242, 6032243, 6032304, 6032305, 6032306,
       6033530, 6034502, 6034511, 6034513, 6035379, 6035380, 6035381,
       6035382, 6035383, 6035384, 6035434, 6035435, 6037705, 6037706,
       6037707, 6039538, 6039539, 6039543, 6039544, 6039545, 6039551,
       6039552, 6039553, 6039554, 6039555, 6039556, 6039891, 6039892,
       6039893, 6039

In [130]:
route_5_trip_6025741 = route_5.loc[(route_5['trip_id'] == 6025741)] #selected trip_id 6025741

#I trimmed gtsf data down to departure_time, stop_id, stop_sequence, stop_lat_, stop_lon
#Just for easier visuals
route_5_trip_6025741[['departure_time', 'stop_id', 'stop_sequence', 'direction_id', 'stop_lat', 'stop_lon']]

departure_time  stop_id  stop_sequence  direction_id   stop_lat  \
68208       05:55:00   903009              1             1  33.920969   
68923       05:57:31   902999              2             1  33.918901   
69218       05:58:00   902704              3             1  33.918961   
69513       05:59:05   903033              4             1  33.919048   
69808       06:00:00   902868              5             1  33.919113   
70103       06:00:51   903199              6             1  33.919057   
70398       06:02:15   902746              7             1  33.922431   
70594       06:02:55   906333              8             1  33.923119   
70790       06:03:56   902662              9             1  33.923710   
70986       06:04:21   903157             10             1  33.924333   
71182       06:04:48   903158             11             1  33.924741   
71387       06:05:18   902701             12             1  33.924189   
71690       06:05:44   902700             13             1  33.922703   
71984       06:06:29   906334             14             1  33.920108   
72180       06:07:00   902751             15             1  33.918641   
72376       06:07:38   906361             16             1  33.916242   
72572       06:08:17   902749             17             1  33.913720   
72768       06:09:31   902699             18             1  33.909002   
72964       06:09:58   902698             19             1  33.907231   
73160       06:10:26   902697             20             1  33.905396   
73356       06:11:07   902696             21             1  33.902783   
73552       06:11:26   902747             22             1  33.901628   
73748       06:11:54   905744             23             1  33.900208   
73944       06:12:38   905743             24             1  33.897528   
74140       06:13:17   905814             25             1  33.895022   
74336       06:13:58   905869             26             1  33.892400   
74532       06:14:33   905864             27             1  33.890163   
74728       06:15:18   905863             28             1  33.887263   
74924       06:16:05   905815             29             1  33.884418   
75120       06:16:27   905746             30             1  33.883138   
75316       06:17:30   905812             31             1  33.879156   
75512       06:18:06   905811             32             1  33.876883   
75708       06:18:25   905626             33             1  33.875746   
75904       06:19:08   905580             34             1  33.873007   
76100       06:19:49   905644             35             1  33.870358   
76296       06:20:33   905763             36             1  33.867530   
76492       06:21:17   905762             37             1  33.864746   
76688       06:21:53   905647             38             1  33.862509   
76884       06:22:14   905646             39             1  33.861167   
77080       06:22:27   905645             40             1  33.860334   
77276       06:23:08   906001             41             1  33.857882   
77472       06:24:28   905928             42             1  33.853317   
77668       06:25:13   905760             43             1  33.851152   
77864       06:25:51   905408             44             1  33.849614   
78060       06:26:32   905409             45             1  33.848452   
78256       06:27:08   905410             46             1  33.846510   
78452       06:28:00   905526             47             1  33.844399   
78648       06:29:24   901705             48             1  33.841710   
78844       06:30:34   900481             49             1  33.839531   
79040       06:31:52   900482             50             1  33.837061   
79236       06:33:16   900483             51             1  33.834197   
79432       06:34:35   901706             52             1  33.831503   
79628       06:35:29   901707             53             1  33.829650   
79824       06:36:18   901755             54             1  33.8

In [131]:
first_and_last_rows = route_5_trip_6025741.iloc[[0,-1]]
first_and_last_rows

trip_id arrival_time departure_time  stop_id  stop_sequence  \
68208  6025741     05:55:00       05:55:00   903009              1   
80216  6025741     06:40:00       06:40:00   902145             56   

       direction_id  block_id route_short_name  service_id   stop_lat  \
68208             1    328963                5           3  33.920969   
80216             1    328963                5           3  33.823072   

        stop_lon  
68208 -84.344734  
80216 -84.369950

In [132]:
#In the next 2 cells, I tried determining direction by seeing whether vertical distance or horizontal distance was longer. 
#Turns out that is a poor way to indicate direction of service.

#I ended up determining direction by mapping gtsf table above into Tableau. 
#I pasted Route 5 Trip 6025741 into Weekly Progress Google Slides.
#The first stop is at Dunwoody Station.
#The last stop is at Lindbergh Bus Station.

In [133]:
total_horizontal_distance = route_5_trip_6025741.iloc[0]['stop_lat'] - route_5_trip_6025741.iloc[-1]['stop_lat']
total_horizontal_distance
#latitude wise: negative --> positive 
#               west -----> east
#         33.823072 -----> 33.920969
#so, if lat[0] - lat[-1] > 0, it is east to west, so westbound
#in this example, (33.92) - (33.82), it is westbound

0.09789700000000323

In [134]:
total_vertical_distance = route_5_trip_6025741.iloc[0]['stop_lon'] - route_5_trip_6025741.iloc[-1]['stop_lon']
total_vertical_distance
#longitude wise: negative ---> positive
#                south ------> north
#           -84.369950 ------> -84.344734
#if long[0] - long[-1] > 0, it is north to south, so southbound
#in this example, (-84.344734) - (-84.369950), it is southbound

#I thought if I could determine if the vertical distance or the horizontal distance was longer,
#I could determine direction without relying on direction_id but in this example, the actual trip is southbound
#but it technically has a larger horizontal distance....help me...

0.02521600000000035

In [135]:
#assign variable to the scheduled departure time
scheduled_time = route_5_trip_6025741.iloc[0]["departure_time"]

In [136]:
#Because APC is such a big file to work with,
#I used the code below to "chunk" APC into 111 csv files with 2,000,000 rows each
#Ideally, I want to be able to run this code on the entire APC.csv
#You can find this code labelled as chunk.py in this Github repository.

#import pandas as pd

#chunk_size = 2000000
#batch_no = 1

#for chunk in pd.read_csv('APC.csv', chunksize = chunk_size):
#	chunk.to_csv('APC' + str(batch_no) + '.csv', index = False)
#	batch_no += 1

In [137]:
#I'm going to try matching 1 row APC1.csv with a gtsf row
APC1 = pd.read_csv("APC1.csv", delimiter = ",")
APC1.columns

Index(['CALENDAR_ID', 'ROUTE_ABBR', 'ROUTE_NAME', 'RUN_NUM', 'BLOCK_ABBR',
       'BLOCK_NUM', 'TRIP_SEQUENCE', 'TRIP_SERIAL_NUMBER', 'BLOCK_TRIP_SEQ',
       'ROUTE_DIRECTION_NAME', 'SERVICE_TYPE_TEXT', 'VECHILE_TAG',
       'BLOCK_STOP_ORDER', 'LATITUDE', 'LONGITUDE', 'ARRIVAL_TIME',
       'ARRIVAL_TM_HM', 'DEPARTURE_TIME', 'DEPARTURE_TM_HM', 'BOARD',
       'ALIGHT'],
      dtype='object')

In [138]:
#In order to reduce the number of rows to compare, I sliced the dataframe to include:
#route 5
#southbound
#departure times between hours 4 and 7 (1 hour before start time and 1 hour after final time)

gtsf_std = route_5_trip_6025741.iloc[0]['departure_time'][0:2]
gtsf_std = int(gtsf_std)
#print(gtsf_std) #5
gtsf_ftd = route_5_trip_6025741.iloc[-1]['departure_time'][0:2]
gtsf_ftd = int(gtsf_ftd)
#print(gtsf_ftd) #6


adjusted_APC = APC1.loc[(APC1['ROUTE_ABBR'] == 5) & (APC1['ROUTE_DIRECTION_NAME'] == "Southbound") & 
         ((APC1['ARRIVAL_TM_HM']).str.contains("0" + str(gtsf_std-1) +":") 
          | (APC1['ARRIVAL_TM_HM']).str.contains("0" + str(gtsf_std) +":")
          | (APC1['ARRIVAL_TM_HM']).str.contains("0" + str(gtsf_std+1) +":")
          | (APC1['DEPARTURE_TM_HM']).str.contains("0" + str(gtsf_ftd-1) +":")
          | (APC1['DEPARTURE_TM_HM']).str.contains("0" + str(gtsf_ftd) +":")
          | (APC1['DEPARTURE_TM_HM']).str.contains("0" + str(gtsf_ftd+1) +":"))]
adjusted_APC

CALENDAR_ID  ROUTE_ABBR                     ROUTE_NAME  \
1544    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
1545    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
1546    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
1547    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
1548    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
1549    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
1550    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
1551    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
2099    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
2100    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
3218    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
3219    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
3220    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
3225    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
3822    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
3823    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
3824    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
3825    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
3826    2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
12856   2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
12857   2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
12858   2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
12859   2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
13350   2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
13351   2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
13352   2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
13353   2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
13354   2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
13355   2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
13356   2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs   
...                     ...         ...                            ...   
193977  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
193978  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
193979  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
193980  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
194004  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
194005  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
194006  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
194007  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
194008  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
194009  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
194010  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
194014  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
194015  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
194017  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
194018  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
194019  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
194021  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
194022  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
194690  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
195420  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
195421  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
195422  2018-01-04 00:00:00           5  Piedmont Road / Sandy Springs   
195423  2018-01-04 0

In [139]:
#To further slice my dataframe, I wanted to explore VECHILE_TAG since it represents a bus. 
#I have good chances of picking out a sinlge trip that matches route 5's 5:55 - 6:40 southbound trip

for item in adjusted_APC.VECHILE_TAG.unique():
    print(item, adjusted_APC[adjusted_APC['VECHILE_TAG'] == item].TRIP_SERIAL_NUMBER.unique())

1653 [4175512]
1606 [4175515 4193730 4193738]
1671 [4175513 4175519]
1475 [4175511 4175517]
1452 [4175510 4175516]
1669 [4175514]
1562 [4193727 4193735]
1672 [4193728 4193736]
1646 [4193726 4193734 4193742]
1578 [4193729 4193737 4193728 4193736]
1568 [4193731 4193739]
1581 [4193732 4193740 4193729 4193737]
1455 [4193726 4193734]
1650 [4193735 4193727]
2529 [4193741 4193733]
1645 [4193731 4193739]
1570 [4193732 4193740]
1602 [4193727 4193735]
1603 [4193736 4193728]
1604 [4193729 4193737]
1589 [4193726 4193734]


In [158]:
#Here is a noteworthy sample of VECHILE_TAG and TRIP_SERIAL_NUMBER:
bus_1653_trip_4175512 = adjusted_APC.loc[(adjusted_APC['VECHILE_TAG'] == 1653) 
                                         & (adjusted_APC['TRIP_SERIAL_NUMBER'] == 4175512)] 
bus_1653_trip_4175512.sort_values(by=['DEPARTURE_TM_HM'])


CALENDAR_ID  ROUTE_ABBR                     ROUTE_NAME  RUN_NUM  \
1544  2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs      503   
1545  2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs      503   
1546  2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs      503   
1547  2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs      503   
1548  2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs      503   
1549  2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs      503   
1550  2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs      503   
1551  2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs      503   
2099  2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs      503   
2100  2018-01-01 00:00:00           5  Piedmont Road / Sandy Springs      503   

     BLOCK_ABBR  BLOCK_NUM  TRIP_SEQUENCE  TRIP_SERIAL_NUMBER  BLOCK_TRIP_SEQ  \
1544        5-3        NaN          21300             4175512               1   
1545        5-3        NaN          21300             4175512               1   
1546        5-3        NaN          21300             4175512               1   
1547        5-3        NaN          21300             4175512               1   
1548        5-3        NaN          21300             4175512               1   
1549        5-3        NaN          21300             4175512               1   
1550        5-3        NaN          21300             4175512               1   
1551        5-3        NaN          21300             4175512               1   
2099        5-3        NaN          21300             4175512               1   
2100        5-3        NaN          21300             4175512               1   

     ROUTE_DIRECTION_NAME  ... VECHILE_TAG  BLOCK_STOP_ORDER   LATITUDE  \
1544           Southbound  ...        1653                19  33.911470   
1545           Southbound  ...        1653                19  33.911470   
1546           Southbound  ...        1653                19  33.911470   
1547           Southbound  ...        1653                19  33.911470   
1548           Southbound  ...        1653                36  33.871367   
1549           Southbound  ...        1653                43  33.852892   
1550           Southbound  ...        1653                43  33.852892   
1551           Southbound  ...        1653                51  33.837130   
2099           Southbound  ...        1653                56  33.825983   
2100           Southbound  ...        1653                57  33.823313   

      LONGITUDE  ARRIVAL_TIME  ARRIVAL_TM_HM DEPARTURE_TIME  DEPARTURE_TM_HM  \
1544 -84.379308       22162.0          06:09        22176.0            06:09   
1545 -84.379308       22262.0          06:11        22279.0            06:11   
1546 -84.379308       22408.0          06:13        22453.0            06:14   
1547 -84.379308       22537.0          06:15        22549.0            06:15   
1548 -84.381057       22614.0          06:16        22634.0            06:17   
1549 -84.381583       22940.0          06:22        22950.0            06:22   
1550 -84.381583       22979.0          06:22        23173.0            06:26   
1551 -84.368228       23223.0          06:27        23683.0            06:34   
2099 -84.367252       23740.0          06:35        23773.0            06:36   
2100 -84.369579       23832.0          06:37        25045.0            06:57   

     BOARD  ALIGHT  
1544     1       0  
1545     1       0  
1546     2       0  
1547     0       1  
1548     1       0  
1549     0       1  
1550     1       1  
1551     1       1  
2099     0       1  
2100     0       5  

[10 rows x 21 columns]

In [171]:
bus_1653_trip_4175512['STOP_LAT'] = 0.0
bus_1653_trip_4175512['STOP_LONG'] = 0.0

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [174]:
for index,rows in bus_1653_trip_4175512.iterrows():
    APC_lat = rows['LATITUDE']
    APC_lon = rows['LONGITUDE']
    for index,rows in route_5_trip_6025741.iterrows():
        lat = rows['stop_lat']
        lon = rows['stop_lon']
        
        final = [0,0,0,0.0014142136,0] #lat, long, minimum distance, index
        distance = math.sqrt(((lat-APC_lat) ** 2) + ((lon-APC_lon) ** 2))
        lat_diff = APC_lat - lat
        if distance < final[2]:
            final[2] = distance
            final[0] = lat
            final[1] = lon
            final[3] = index
        
        #bus_1653_trip_4175512[index]['STOP_LAT']
        #bus_1653_trip_4175512[index]['STOP_LONG']
        # Does anyone know how to access a cell in a pandas data frame with index and column name?

KeyError: 68208

In [57]:
#pull 1st APC1.csv coordinate and compared to every gtsf coordinate in route_5_trip_602741
APC_lat = adjusted_APC.iloc[0]['LATITUDE'] 
APC_lon = adjusted_APC.iloc[0]['LONGITUDE']

#create an empty pandas dataframe and empty lists for index, stop_lat, stop_lon, lat_diff, lon_diff
time_difference = pd.DataFrame(columns = None)
indexList = []
stop_lat = []
stop_lon = []
lat_diff = []
lon_diff = []

for index,rows in route_5_trip_6025741.iterrows():
    indexList.append(index) 
    stop_lat.append(rows['stop_lat'])
    stop_lon.append(rows['stop_lon'])
    lat_diff.append(abs(APC_lat-rows['stop_lat']))
    lon_diff.append(abs(APC_lon-rows['stop_lon']))

df['index'] = indexList
df['stop_lat'] = stop_lat
df['stop_lon'] = stop_lon
df['lat_diff'] = lat_diff
df['lon_diff'] = lon_diff

df.drop(columns = "min_lat_diff")

index   stop_lat   stop_lon  lat_diff  lon_diff
0   68208  33.920969 -84.344734  0.009499  0.034574
1   68923  33.918901 -84.349888  0.007431  0.029420
2   69218  33.918961 -84.351952  0.007491  0.027356
3   69513  33.919048 -84.356509  0.007578  0.022799
4   69808  33.919113 -84.360336  0.007643  0.018972
5   70103  33.919057 -84.363941  0.007587  0.015367
6   70398  33.922431 -84.367997  0.010961  0.011311
7   70594  33.923119 -84.370666  0.011649  0.008642
8   70790  33.923710 -84.374880  0.012240  0.004428
9   70986  33.924333 -84.376452  0.012863  0.002856
10  71182  33.924741 -84.378245  0.013271  0.001063
11  71387  33.924189 -84.379132  0.012719  0.000176
12  71690  33.922703 -84.379218  0.011233  0.000090
13  71984  33.920108 -84.379042  0.008638  0.000266
14  72180  33.918641 -84.379085  0.007171  0.000223
15  72376  33.916242 -84.379236  0.004772  0.000072
16  72572  33.913720 -84.379550  0.002250  0.000242
17  72768  33.909002 -84.378784  0.002468  0.000524
18  72964  33.907231 -84.378718  0.004239  0.000590
19  73160  33.905396 -84.378695  0.006074  0.000613
20  73356  33.902783 -84.378606  0.008687  0.000702
21  73552  33.901628 -84.379160  0.009842  0.000148
22  73748  33.900208 -84.380478  0.011262  0.001170
23  73944  33.897528 -84.381554  0.013942  0.002246
24  74140  33.895022 -84.381388  0.016448  0.002080
25  74336  33.892400 -84.381190  0.019070  0.001882
26  74532  33.890163 -84.381524  0.021307  0.002216
27  74728  33.887263 -84.381677  0.024207  0.002369
28  74924  33.884418 -84.380749  0.027052  0.001441
29  75120  33.883138 -84.380128  0.028332  0.000820
30  75316  33.879156 -84.379852  0.032314  0.000544
31  75512  33.876883 -84.379971  0.034587  0.000663
32  75708  33.875746 -84.380514  0.035724  0.001206
33  75904  33.873007 -84.381089  0.038463  0.001781
34  76100  33.870358 -84.381109  0.041112  0.001801
35  76296  33.867530 -84.381151  0.043940  0.001843
36  76492  33.864746 -84.380757  0.046724  0.001449
37  76688  33.862509 -84.380276  0.048961  0.000968
38  76884  33.861167 -84.380450  0.050303  0.001142
39  77080  33.860334 -84.380558  0.051136  0.001250
40  77276  33.857882 -84.381481  0.053588  0.002173
41  77472  33.853317 -84.381663  0.058153  0.002355
42  77668  33.851152 -84.379376  0.060318  0.000068
43  77864  33.849614 -84.377089  0.061856  0.002219
44  78060  33.848452 -84.374182  0.063018  0.005126
45  78256  33.846510 -84.372831  0.064960  0.006477
46  78452  33.844399 -84.371807  0.067071  0.007501
47  78648  33.841710 -84.370530  0.069760  0.008778
48  78844  33.839531 -84.369377  0.071939  0.009931
49  79040  33.837061 -84.368146  0.074409  0.011162
50  79236  33.834197 -84.368046  0.077273  0.011262
51  79432  33.831503 -84.368088  0.079967  0.011220
52  79628  33.829650 -84.367782  0.081820  0.011526
53  79824  33.828013 -84.367303  0.083457  0.012005
54  80020  33.825981 -84.367215  0.085489  0.012093
55  80216  33.823072 -84.369950  0.088398  0.009358

In [110]:
#This was not used in this Jupyter Notebook, but if APC is split into 111 csv files, 
#this was a way I found to parse through files in the same directory
import os
directory = r'C:\Users\Vanessa Ahn\Downloads\Marta VIP\180314\chunkAPC'
for filename in os.listdir(directory):
    print(filename)
    #APC = pd.read_csv(filename, delimiter=",")

APC1.csv
APC10.csv
APC100.csv
APC101.csv
APC102.csv
APC103.csv
APC104.csv
APC105.csv
APC106.csv
APC107.csv
APC108.csv
APC109.csv
APC11.csv
APC110.csv
APC111.csv
APC12.csv
APC13.csv
APC14.csv
APC15.csv
APC16.csv
APC17.csv
APC18.csv
APC19.csv
APC2.csv
APC20.csv
APC21.csv
APC22.csv
APC23.csv
APC24.csv
APC25.csv
APC26.csv
APC27.csv
APC28.csv
APC29.csv
APC3.csv
APC30.csv
APC31.csv
APC32.csv
APC33.csv
APC34.csv
APC35.csv
APC36.csv
APC37.csv
APC38.csv
APC39.csv
APC4.csv
APC40.csv
APC41.csv
APC42.csv
APC43.csv
APC44.csv
APC45.csv
APC46.csv
APC47.csv
APC48.csv
APC49.csv
APC5.csv
APC50.csv
APC51.csv
APC52.csv
APC53.csv
APC54.csv
APC55.csv
APC56.csv
APC57.csv
APC58.csv
APC59.csv
APC6.csv
APC60.csv
APC61.csv
APC62.csv
APC63.csv
APC64.csv
APC65.csv
APC66.csv
APC67.csv
APC68.csv
APC69.csv
APC7.csv
APC70.csv
APC71.csv
APC72.csv
APC73.csv
APC74.csv
APC75.csv
APC76.csv
APC77.csv
APC78.csv
APC79.csv
APC8.csv
APC80.csv
APC81.csv
APC82.csv
APC83.csv
APC84.csv
APC85.csv
APC86.csv
APC87.csv
APC88.csv
APC89.